In [1]:
import pandas as pd
battle_data = pd.read_parquet("data/battles.parquet")
print(battle_data.columns)
print(battle_data.shape)

Index(['format', 'players', 'log', 'formatid', 'rating', 'inputlog', 'winner',
       'loser'],
      dtype='object')
(40959, 8)


In [2]:
# Find all unique actions that potentially needs to be accounted for
unique_actions = set()
battle_data.log.apply(lambda x: unique_actions.update([action.split("|")[1] for event in x for action in event if len(action.split("|")) > 1]))
unique_actions

{'',
 '-ability',
 '-activate',
 '-anim',
 '-block',
 '-boost',
 '-clearallboost',
 '-clearboost',
 '-clearnegativeboost',
 '-closer',
 '-crit',
 '-curestatus',
 '-damage',
 '-end',
 '-enditem',
 '-fail',
 '-fieldend',
 '-fieldstart',
 '-formechange',
 '-heal',
 '-hint',
 '-hitcount',
 '-immune',
 '-item',
 '-message',
 '-miss',
 '-mustrecharge',
 '-prepare',
 '-resisted',
 '-setboost',
 '-sethp',
 '-sideend',
 '-sidestart',
 '-singlemove',
 '-singleturn',
 '-start',
 '-status',
 '-supereffective',
 '-swapsideconditions',
 '-terastallize',
 '-transform',
 '-unboost',
 '-weather',
 'cant',
 'chat',
 'detailschange',
 'drag',
 'error',
 'faint',
 'gen',
 'hidelines',
 'inactiveoff',
 'j',
 'move',
 'n',
 'rated',
 'replace',
 'rule',
 'switch',
 'teamsize',
 'tier'}

In [3]:
def pretty_convert_dict(d, indent=0):
    result = ""
    for key, value in d.items():
        if value is None or value == "" or value == [] or value == {}:
            continue
        result += "\t" * indent + " " + str(key) + "\n"
        if isinstance(value, dict):
            result += pretty_convert_dict(value, indent + 1)
        else:
            result += "\t" * (indent + 1) + " " + str(value) + "\n"
    return result

In [4]:
move_data = pd.read_csv("data/moves.csv")
move_data.head()

,name,type,category,power,accuracy,pp,effect
0,"10,000,000 Volt Thunderbolt",Electric,special,195.0,NaN,1,Pikachu-exclusive Z-Move. High critical hit ra...
1,Absorb,Grass,special,20.0,100.0,25,User recovers half the HP inflicted on opponent.
2,Accelerock,Rock,physical,40.0,100.0,20,User attacks first.
3,Acid,Poison,special,40.0,100.0,30,May lower opponent's Special Defense.
4,Acid Armor,Poison,status,NaN,NaN,20,Sharply raises user's Defense.


In [5]:
def get_team_data(player_team: dict, side: str, turn_data):
    # turn data is one nested list in the log column field
    for event in turn_data:
        for action in event:
            split_action = action.split("|")
            # Detect new pokemon based on switch or drag
            if action.startswith("|switch|{}:".format(side)) or action.startswith(
                "|drag|{}:".format(side)
            ):
                # clear out the boosts of every pokemon
                for key in player_team.keys():
                    player_team[key]["boosts"] = {}
                pokemon_key = split_action[2].split(": ")[1]
                pokemon_data = split_action[3].split(", ")
                pokemon_name = pokemon_data[0]
                # print("KEY: {}, NAME: {}".format(pokemon_key, pokemon_name))
                hp = int(split_action[4].split("/")[0])
                if pokemon_key in player_team:
                    player_team[pokemon_key]["hp"] = hp
                    continue
                else:

                    player_team[pokemon_key] = {}
                    # create an empty moves set
                    player_team[pokemon_key]["moves"] = set()
                    # set hp
                    player_team[pokemon_key]["hp"] = hp
                    # set max hp
                    player_team[pokemon_key]["boosts"] = {}
                # set name
                player_team[pokemon_key]["name"] = pokemon_name
                maximum_hp = int(split_action[4].split("/")[1].split(" ")[0])
                player_team[pokemon_key]["maximum hp"] = maximum_hp
                # Remove the L from the level
                player_team[pokemon_key]["level"] = (
                    pokemon_data[1][1:] if len(pokemon_data) > 1 else None
                )
                # If gender is not specified, set it to None
                player_team[pokemon_key]["gender"] = (
                    pokemon_data[2] if len(pokemon_data) > 2 else None
                )

            elif action.startswith("|move|{}:".format(side)):
                # parse out the pokemon name, make sure to remove the player name, since the move is going to be formatted as player: move
                pokemon_key = split_action[2].split(": ")[1]
                move_name = split_action[3]
                player_team[pokemon_key]["moves"].add(move_name)
            elif action.startswith("|-boost|{}:".format(side)):
                boost_type = split_action[3]
                boost_amount = int(split_action[4])
                # Check if boost type is in the pokemon dictionary
                pokemon_key = split_action[2][2 + len(side) :]
                if boost_type in player_team[pokemon_key]["boosts"]:
                    player_team[pokemon_key]["boosts"][boost_type] += boost_amount
                else:
                    player_team[pokemon_key]["boosts"][boost_type] = boost_amount
            elif action.startswith("|-unboost|{}:".format(side)):
                # parse out the boost information
                boost_type = split_action[3]
                boost_amount = int(split_action[4])
                # Check if boost type is in the pokemon dictionary
                if boost_type in player_team[pokemon_key]["boosts"]:
                    player_team[pokemon_key]["boosts"][boost_type] -= boost_amount
                else:
                    player_team[pokemon_key]["boosts"][boost_type] = -boost_amount
            elif action.startswith("|-setboost|{}:".format(side)):
                # parse out the boost information
                boost_type = split_action[3]
                boost_amount = int(split_action[4])
                player_team[pokemon_key]["boosts"][boost_type] = boost_amount
            elif action.startswith("|-heal|{}:".format(side)):
                pokemon_key = split_action[2][2 + len(side) :]
                health = int(split_action[3].split("/")[0])
                player_team[pokemon_key]["hp"] = health
            elif action.startswith("|-damage|{}:".format(side)):
                pokemon_key = split_action[2][2 + len(side) :]
                if split_action[3] == "0 fnt":
                    player_team[pokemon_key]["hp"] = 0
                else:
                    health = int(split_action[3].split("/")[0])
                    player_team[pokemon_key]["hp"] = health
            elif action.startswith("|-status|{}:".format(side)):
                pokemon_key = split_action[2][2 + len(side) :]
                status = split_action[3]
                player_team[pokemon_key]["status"] = status
            elif action.startswith("|-curestatus|{}:".format(side)):
                pokemon_key = split_action[2][2 + len(side) :]
                player_team[pokemon_key]["status"] = None
            elif action.startswith("|-terastallize|{}:".format(side)):
                pokemon_key = split_action[2][2 + len(side) :]
                player_team[pokemon_key]["tera"] = split_action[3]
            elif action.startswith("|detailschange|{}:".format(side)):
                pokemon_key = split_action[2].split(":")[1][1:]
                new_pokemon_name = split_action[3].split(", ")[0]
                if "Zoroak" in new_pokemon_name or "Ditto" in new_pokemon_name:
                    player_team[new_pokemon_name] = player_team.pop(pokemon_key)
                else:
                    player_team[pokemon_key]["name"] = new_pokemon_name
            elif action.startswith("|replace|{}:".format(side)):
                replacing_with_pkm_key = split_action[2].split(": ")[1]
                replacing_with_pkm_name = split_action[3].split(", ")[0]
                to_be_replaced_pkm_key = ""
                # find the pokemon that is being replaced
                for forward_action in event:
                    if forward_action.startswith("|-damage|{}:".format(side)):
                        to_be_replaced_pkm_key = forward_action.split("|")[2][
                            2 + len(side) :
                        ]
                        break
                # replace the pokemon
                player_team[replacing_with_pkm_key] = player_team[
                    to_be_replaced_pkm_key
                ]
                player_team[replacing_with_pkm_key]["name"] = replacing_with_pkm_name
                # clear out the attributes of the replaced pokemon
                player_team[to_be_replaced_pkm_key]["moves"] = set()
                player_team[to_be_replaced_pkm_key]["boosts"] = {}
            elif action.startswith("|-clearallboost"):
                for key in player_team.keys():
                    player_team[key]["boosts"] = {}
            elif action.startswith("|-clearnegativeboost|{}:".format(side)):
                for key in player_team.keys():
                    for boost_type in list(player_team[key]["boosts"].keys()):
                        if player_team[key]["boosts"][boost_type] < 0:
                            player_team[key]["boosts"][boost_type] = 0
            elif action.startswith("|-clearboost|{}:".format(side)):
                pokemon_key = split_action[2][2 + len(side) :]
                player_team[pokemon_key]["boosts"] = {}
            elif action.startswith("|-sethp|{}:".format(side)):
                pokemon_key = split_action[2][2 + len(side) :]
                player_team[pokemon_key]["hp"] = int(split_action[3].split("/")[0])


    return player_team

In [6]:
import requests

random_sets = requests.get(
    "https://pkmn.github.io/randbats/data/gen9randombattle.json"
).json()
print("Set amount: {}".format(len(random_sets)))

Set amount: 506


In [7]:
def find_moveset(team_data, random_sets):
    for pokemon_name in team_data.keys():
        if len(team_data[pokemon_name]["moves"]) == 4:
            continue
        if pokemon_name in random_sets.keys():
            known_moves = team_data[pokemon_name]["moves"]
            possible_sets = random_sets[pokemon_name]["roles"]
            for role in possible_sets:
                if isinstance(known_moves, dict):
                    known_moves = set(known_moves.keys())
                if known_moves.issubset(possible_sets[role]["moves"]):
                    # also grab the evs and ivs for the pokemon
                    if "evs" in possible_sets[role]:
                        team_data[pokemon_name]["evs"] = possible_sets[role]["evs"]
                    if "ivs" in possible_sets[role]:
                        team_data[pokemon_name]["ivs"] = possible_sets[role]["ivs"]

                    potential_moveset = possible_sets[role]["moves"]
                    seen_unseen_moves = dict()
                    for move in potential_moveset:
                        if move in known_moves:
                            seen_unseen_moves[move] = "seen"
                        else:
                            seen_unseen_moves[move] = "unseen"
                    team_data[pokemon_name]["moves"] = seen_unseen_moves

                    break
    return team_data

In [8]:
from javascript import require


def calculate_damage(atkr: dict, defdr: dict, move_used):
    damage_calc = require("@smogon/calc")
    generation = damage_calc.Generations.get(9)
    attacker = None
    defender = None
    atkr_attributes = {}
    if "level" in atkr:
        atkr_attributes["level"] = atkr.get("level")
    if "item" in atkr:
        atkr_attributes["item"] = atkr.get("item")
    if "boosts" in atkr:
        atkr_attributes["boosts"] = atkr.get("boosts")
    if "tera" in atkr:
        atkr_attributes["teraType"] = atkr.get("tera")
    if "item" in atkr:
        atkr_attributes["item"] = atkr.get("item")
    if "evs" in atkr:
        atkr_attributes["evs"] = atkr.get("evs")
    if "ivs" in atkr:
        atkr_attributes["ivs"] = atkr.get("ivs")
    defdr_attributes = {}
    if "level" in defdr:
        defdr_attributes["level"] = defdr.get("level")
    if "item" in defdr:
        defdr_attributes["item"] = defdr.get("item")
    if "boosts" in defdr:
        defdr_attributes["boosts"] = defdr.get("boosts")
    if "tera" in defdr:
        defdr_attributes["teraType"] = defdr.get("tera")
    if "item" in defdr:
        defdr_attributes["item"] = defdr.get("item")
    if "evs" in defdr:
        defdr_attributes["evs"] = defdr.get("evs")
    if "ivs" in defdr:
        defdr_attributes["ivs"] = defdr.get("ivs")
    try:
        attacker = damage_calc.Pokemon.new(
            generation, atkr.get("name"), atkr_attributes
        )
    except:
        attacker = damage_calc.Pokemon.new(
            generation, atkr.get("name").split("-")[0], atkr_attributes
        )
    try:
        defender = damage_calc.Pokemon.new(
            generation, defdr.get("name"), defdr_attributes
        )
    except:
        defender = damage_calc.Pokemon.new(
            generation, defdr.get("name").split("-")[0], defdr_attributes
        )
    move = damage_calc.Move.new(generation, move_used)
    result = damage_calc.calculate(generation, attacker, defender, move)
    if result.damage == 0:
        return 0, 0
    if isinstance(result.damage, str):
        return result.damage + "%", result.damage + "%"
    dmg_range = result.damage.valueOf()
    min_dmg = min(dmg_range)
    max_dmg = max(dmg_range)
    # calculate the percentage of damage
    hp = defdr.get("hp")
    if hp == 0:
        return "100%", "100%"
    if hp == None:
        hp = defdr.get("maximum hp")
    min_dmg_percent = int(min_dmg / hp * 100)
    max_dmg_percent = int(max_dmg / hp * 100)
    return str(min_dmg_percent) + "%", str(max_dmg_percent) + "%"

In [9]:
def get_move_effect_on_team(
    atk_pkm_name: str, atk_team: dict, def_team: dict, def_name: str
):
    move_effect = {}
    # find the moves of the attacker
    moves = []
    for key in atk_team.keys():
        if atk_pkm_name in key or key in atk_pkm_name:
            moves = atk_team[key]["moves"]
            break
    if len(moves) == 0:
        return move_effect
    for move in moves:
        move_effect[move] = {}
        if def_name != "":
            move_effect[move][def_name] = calculate_damage(
                atk_team[atk_pkm_name], def_team[def_name], move
            )
            continue
        for def_pkm_name in def_team.keys():
            move_effect[move][def_pkm_name] = calculate_damage(
                atk_team[atk_pkm_name], def_team[def_pkm_name], move
            )
    return move_effect

In [10]:
def find_current_pokemon(side: str, turn_data):
    # look back through the turn data to find the current pokemon
    for event in reversed(turn_data):
        for action in reversed(event):
            if action.startswith("|switch|{}:".format(side)) or action.startswith(
                "|drag|{}:".format(side)
            ):
                return action.split("|")[2].split(": ")[1]

In [11]:
from typing import List


def parse_player_next_turn(turn: List[str], side):
    # parse to see if the player used a move, swapped, or swapped because of a faint
    # return the move used or the pokemon swapped to
    chosen_action = {}
    move_chosen = False
    pokemon_chosen = False
    pokemon_fainted = False
    for action in turn:
        if action.startswith("|move|{}:".format(side)) and not move_chosen:
            move_chosen = action.split("|")[3]
            chosen_action["move"] = move_chosen
            move_chosen = True
        elif action.startswith("|switch|{}:".format(side)) and not pokemon_chosen:
            pokemon_switched = action.split("|")[3].split(", ")[0]
            chosen_action["switch"] = pokemon_switched
            pokemon_chosen = True
        elif action.startswith("|faint|{}:".format(side)) and not pokemon_fainted:
            pokemon_fainted = action.split("|")[2]
            chosen_action["faint"] = pokemon_fainted
            pokemon_fainted = True
        elif action.startswith("|cant|{}:".format(side)):
            chosen_action["status"] = action.split("|")[3]
    return chosen_action

In [12]:
def produce_prompt(game_history):
    prompt = """You are an expert in Pokemon and competitive battling. You are the best Pokemon showdown player in the random battle format.
You have been invited to a Pokemon tournament with a grand prize of $1,000,000. You are confident that you will win the tournament.
You're skills are the best in the world, and you have never lost a random battle in your life. You are the best of the best.
You know when to switch, when to set up, and when to attack. You know the best moves to use in every situation.

Below is information about your team, what you currently know about the enemy team, and the current history of the battle.

Here is the current history of the battle:

GAME_HISTORY

Your current team and moves as to the best of your knowledge:

PLAYER_TEAM_INFO

The enemy team and moves as to the best of your knowledge:

ENEMY_TEAM_INFO

For the enemy, you know that random battles uses some given sets of moves per Pokemon. Their potential moves could be the following:

ENEMY_TEAM_POKEMON_DB

Given that you currently have sent out PLAYER_POKEMON as your pokemon and the enemy has sent out ENEMY_POKEMON, here is what your moves can probably do to the enemy in terms of hp ranges:

PLAYER_MOVES_IMPACT

Here is a description of your moves and what they do:

PLAYER_MOVES_DESCRIPTION

However, given the potential moves the enemy has, here is what the enemy can probably do to you in terms of hp ranges:

ENEMY_MOVES_IMPACT_INDIVIDUAL

This is also what the enemy moves can probably do to the rest of your team, if you decide to switch, do note if you switch you are giving the enemy a free turn to attack you:

ENEMY_MOVES_IMPACT_TEAM

This is a description of the enemy moves and what they do:

ENEMY_MOVES_DESCRIPTION

Again, right now your pokemon is PLAYER_POKEMON and the enemy pokemon is ENEMY_POKEMON.

LLM_ACTION

"""
    faint_prompt_addition = """
Your pokemon have fainted.
Given the rest of your team:

PLAYER_TEAM_LEFT

SWITCH_PKMN

POKEMON_CHOSEN
"""
    switch_prompt_addition = """
SWITCH_PKMN

POKEMON_CHOSEN
"""
    use_move_prompt_addition = """
CHOSE_MOVE

MOVE_CHOSEN
"""
    status_prompt_addition = """
Due to the status condition:

STATUS_CONDITION

You were unable to use the move.
"""
    values = {}
    player_team_info = get_team_data({}, "winner", game_history)
    for turn in range(1, len(game_history)):
        # clear out the boosts and hps from the player_team_info
        for key in player_team_info.keys():
            if "hp" in player_team_info[key]:
                player_team_info[key].pop("hp", None)
            if "boosts" in player_team_info[key]:
                player_team_info[key].pop("boosts", None)
            if "status" in player_team_info[key]:
                player_team_info[key].pop("status", None)
        turn_data = game_history[:turn]
        player_state = get_team_data({}, "winner", turn_data)
        # extract unseen pokemon and current pokemon moves from player_team_info and merge into player_state
        for key in player_team_info.keys():
            if key not in player_state.keys():
                player_state[key] = player_team_info[key]
            else:
                player_state[key]["moves"] = player_team_info[key]["moves"]
        enemy_team_info = get_team_data({}, "loser", turn_data)
        enemy_team_pokemon_db = find_moveset(enemy_team_info, random_sets)
        # filter out the pokemon that have fainted
        player_team_alive = {
            key: value
            for key, value in player_state.items()
            if value.get("hp", 0) > 0 or value.get("hp") == None
        }
        enemy_team_alive = {
            key: value
            for key, value in enemy_team_info.items()
            if value.get("hp", 0) > 0 or value.get("hp") == None
        }

        player_sent_out = find_current_pokemon("winner", turn_data)
        enemy_sent_out = find_current_pokemon("loser", turn_data)

        player_moves_impact = get_move_effect_on_team(
            player_sent_out, player_team_alive, enemy_team_alive, ""
        )
        enemy_moves_impact = get_move_effect_on_team(
            enemy_sent_out, enemy_team_alive, player_team_alive, player_sent_out
        )
        enemy_moves_impact_team = get_move_effect_on_team(
            enemy_sent_out, enemy_team_alive, player_team_alive, ""
        )

        player_move_description = {}
        for move in player_team_info[player_sent_out]["moves"]:
            player_move_description[move] = move_data[
                move_data["name"] == move
            ].to_dict(orient="records")[0]
        enemy_move_description = {}
        for move in enemy_team_info[enemy_sent_out]["moves"]:
            enemy_move_description[move] = move_data[move_data["name"] == move].to_dict(
                orient="records"
            )[0]

        values[turn] = (
            prompt.replace("PLAYER_TEAM_INFO", pretty_convert_dict(player_team_info))
            .replace("ENEMY_TEAM_INFO", pretty_convert_dict(enemy_team_info))
            .replace(
                "ENEMY_TEAM_POKEMON_DB", pretty_convert_dict(enemy_team_pokemon_db)
            )
            .replace("PLAYER_POKEMON", player_sent_out)
            .replace("ENEMY_POKEMON", enemy_sent_out)
            .replace("PLAYER_MOVES_IMPACT", pretty_convert_dict(player_moves_impact))
            .replace(
                "PLAYER_MOVES_DESCRIPTION", pretty_convert_dict(player_move_description)
            )
            .replace(
                "ENEMY_MOVES_IMPACT_INDIVIDUAL", pretty_convert_dict(enemy_moves_impact)
            )
            .replace(
                "ENEMY_MOVES_DESCRIPTION", pretty_convert_dict(enemy_move_description)
            )
            .replace(
                "ENEMY_MOVES_IMPACT_TEAM", pretty_convert_dict(enemy_moves_impact_team)
            )
            .replace(
                "GAME_HISTORY",
                "\n\n".join(["\n".join(turn) for turn in game_history[:turn]]),
            )
        )
        if turn + 1 < len(game_history):
            player_next_turn: dict = parse_player_next_turn(
                game_history[turn], "winner"
            )
            if "faint" in player_next_turn:
                # filter out the pokemon that we know is hp 0 from the player_team_info
                player_team_left = {}
                for key in player_team_info.keys():
                    if key in player_state.keys() and player_state[key].get("hp") == 0:
                        continue
                    else:
                        player_team_left[key] = player_team_info[key]
                values[turn] += faint_prompt_addition.replace(
                    "PLAYER_TEAM_LEFT", str(list(player_team_left.keys()))
                ).replace("POKEMON_CHOSEN", player_next_turn["switch"])
            elif "switch" in player_next_turn:
                values[turn] += switch_prompt_addition.replace(
                    "POKEMON_CHOSEN", player_next_turn["switch"]
                )
            if "move" in player_next_turn:
                values[turn] += use_move_prompt_addition.replace(
                    "MOVE_CHOSEN", player_next_turn["move"]
                )
            if "status" in player_next_turn:
                values[turn] += status_prompt_addition.replace(
                    "STATUS_CONDITION", player_next_turn["status"]
                )
    return values

In [13]:
"""
# This is the code block of shame
import numpy as np
replace_func = np.vectorize(lambda x: x.replace("Hounloserom", "Houndoom"))
battle_data.log = battle_data.log.apply(lambda log_data: [ replace_func(event) for event in log_data ])
"""

'\n# This is the code block of shame\nimport numpy as np\nreplace_func = np.vectorize(lambda x: x.replace("Hounloserom", "Houndoom"))\nbattle_data.log = battle_data.log.apply(lambda log_data: [ replace_func(event) for event in log_data ])\n'

In [14]:
from tqdm import tqdm
tqdm.pandas()
sample_prompts = battle_data["log"][:5].progress_apply(produce_prompt)

100%|██████████| 5/5 [00:06<00:00,  1.39s/it]
